# Generalizing Failure Circumstances

One central question in debugging is: _Does this bug occur in other situations, too?_ In this chapter, we present a technique that is set to _generalize_ the circumstances under which a failure occurs. The DDSET algorithm takes a failure-inducing input, breaks it into individual elements. For each element, it tries to find whether it can be replaced by others in the same category, and if so, it _generalizes_ the concrete element to the very category. The result is a _pattern_ that characterizes the failure condition: "The failure occurs for all inputs of the form `(<expr> * <expr>)`.

In [ ]:
from bookutils import YouTubeVideo
# YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* You should have read the [chapter on _delta debugging_](DeltaDebugger.ipynb).

In [ ]:
import bookutils

In [ ]:
import DeltaDebugger

## A Failing Program

As with previous chapters, we use `remove_html_markup()` as our ongoing example. This function is set to remove HTML markup tags (like `<em>`) from a given string `s`, returning the plain text only. We use the version from [the chapter on asssertions](Assertions.ipynb), using an assertion as postcondition checker.

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

For the most inputs, `remove_html_markup()` works just as expected:

In [ ]:
remove_html_markup("Be <em>quiet</em>, he said")

There are inputs, however, for which it fails:

In [ ]:
BAD_INPUT = '<foo>"bar</foo>'

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError(AssertionError):
    remove_html_markup(BAD_INPUT)

In [ ]:
from bookutils import quiz

In contrast to the other chapters, our aim now is not to immediately go and debug `remove_html_markup()`. Instead, we focus on another important question: 

> Under which conditions precisely does `remove_html_markup()` fail?

This question can be generalized to

> What is the set of inputs for which `remove_html_markup()` fails?

Our plan for this is to _generalize_ concrete inputs (such as `BAD_INPUTS`) into an *abstract failure-inducing inputs*. These are patterns formed from a concrete input, but in which specific _placeholders_ indicate sets of inputs that are permitted. In the abstract failure-inducing input

```html
<opening-tag>"bar<closing-tag>
```

for instance, `<opening-tag>` and `<closing-tag>` are placeholders for opening and closing HTML tags, respectively. The pattern indicates that any opening HTML tag and closing HTML tag can be present in the input, as long as the enclosed text reads `"bar`.

Given a concrete failure-inducing input, our aim is to _generalize_ it as much as possible to such an abstract failure-inducing input. The resulting pattern should then

* capture the _circumstances_ under which the program fails;
* allow for _test generation_ by instantiating the placeholders;
* help ensuring our fix is as _general as possible_.

In [ ]:
quiz("What is the value of `out` such that the assertion fails?",
    [
        '`bar`',
        '`bar</foo>`',
        '`"bar</foo>`',
        '`<foo>"bar</foo>`',
    ], '9999999 // 4999999')

## Grammars

To determine abstract failure-inducing inputs, we need means to determine and characterize _sets of inputs_ – known in computer science as _languages_. To formally describe languages, the field of *formal languages* has devised a number of *language specifications* that describe a language.  *Regular expressions* represent the simplest class of these languages to denote sets of strings: The regular expression `[a-z]*`, for instance, denotes a (possibly empty) sequence of lowercase letters.  *Automata theory* connects these languages to automata that accept these inputs; *finite state machines*, for instance, can be used to specify the language of regular expressions.

Regular expressions are great for not-too-complex input formats, and the associated finite state machines have many properties that make them great for reasoning.  To specify more complex inputs, though, they quickly encounter limitations.  At the other end of the language spectrum, we have *universal grammars* that denote the language accepted by *Turing machines*.  A Turing machine can compute anything that can be computed; and with Python being Turing-complete, this means that we can also use a Python program $p$ to specify or even enumerate legal inputs. But then, computer science theory also tells us that each such program has to be written specifically for the input to be considered, which is not the level of automation we want.

The middle ground between regular expressions and Turing machines is covered by *grammars*.  Grammars are among the most popular (and best understood) formalisms to formally specify input languages.  Using a grammar, one can express a wide range of the properties of an input language.  Grammars are particularly great for expressing the *syntactical structure* of an input, and are the formalism of choice to express nested or recursive inputs.  The grammars we use are so-called *context-free grammars*, one of the easiest and most popular grammar formalisms.

A grammar is defined as a mapping of _nonterminal_ symbols (denoted in `<angle brackets>` to lists of alternative _expansions_, which are strings containing _terminal_ symbols and possibly more _nonterminal_ symbols. To make the writing of grammars as simple as possible, we adopt the [fuzzingbook](https://www.fuzzingbook.org/) format that is based on strings and lists.

In [ ]:
import fuzzingbook

Fuzzingbook grammars take the format of a _mapping_ between symbol names and expansions, where expansions are _lists_ of alternatives.  A one-rule grammar for digits thus takes the form

In [ ]:
DIGIT_GRAMMAR = {
    "<start>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

which means that the `<start>` symbol can be expanded into any of the digits listed.

A full grammar for arithmetic expressions looks like this:

In [ ]:
EXPR_GRAMMAR = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["+<factor>",
         "-<factor>",
         "(<expr>)",
         "<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

From such a grammar, one can easily generate inputs that conform to the grammar.

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

In [ ]:
simple_expr_fuzzer = GrammarFuzzer(EXPR_GRAMMAR)

In [ ]:
for i in range(10):
    fuzz_expr = simple_expr_fuzzer.fuzz()
    print(fuzz_expr)

Nonterminals as found in the grammar make natural _placeholders_ in abstract failure-inducing inputs. If we know, for instance, that it is not just the concrete failure-inducing input

```python
(2 * 3)
```

but the abstract failure-inducing input

```html
(<expr> * <expr>)
```

that causes the failure, we immediately see that the error is due to the multiplication operator rather than its operands.

Coming back to our `remove_html_markup()` example, let us create a simple grammar for HTML expressions. A `<html>` element is either plain text or tagged text.

In [ ]:
SIMPLE_HTML_GRAMMAR = {
    "<start>":
        ["<html>"],

    "<html>":
        ["<plain-text>", "<tagged-text>"],
}

Plain text is a simple (possibly empty) sequence of letter, digits, punctuation, and whitespace. (Note how `<plain-text>` is either empty or some character followed by more plain text.) The characters `<` and `>` are not allowed, though.

In [ ]:
import string

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<plain-text>":
        ["", "<plain-char><plain-text>"],

    "<plain-char>":
        ["<letter>", "<digit>", "<other>", "<whitespace>"],

    "<letter>": list(string.ascii_letters),
    "<digit>": list(string.digits),
    "<other>": list(string.punctuation.replace('<', '').replace('>', '')),
    "<whitespace>": list(string.whitespace)
})

Tagged text is a bit more complicated. We have opening tags `<foo>`, followed by some more HTML material, and then closed by a closing tag `</foo>`. (We do not insist that the two tags match.) A self-closing tag has the form `<br/>`. For compatibility reasons, we also allow just opening tags without closing tags, as in `<img>`.

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<tagged-text>":
        ["<opening-tag><html><closing-tag>",
         "<self-closing-tag>",
         "<opening-tag>"],
})

Since the characters `<` and `>` are already reserved for denoting nonterminal symbols, we use the special nonterminal symbols `<lt>` and `<gt>` that expand into `<` and `>`, respectively,

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<opening-tag>":
        ["<lt><id><gt>",
         "<lt><id><attrs><gt>"],

    "<lt>": [ "<" ],
    "<gt>": [ ">" ],

    "<id>":
        ["<letter>", "<id><letter>", "<id><digit>"],

    "<closing-tag>":
        ["<lt>/<id><gt>"],

    "<self-closing-tag>":
        ["<lt><id><attrs>/<gt>"],
})

Finally, HTML tags can have attributes, which are enclosed in quotes.

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<attrs>":
        ["<attr>", "<attr><attrs>" ],

    "<attr>":
        [" <id>='<plain-text>'",
         ' <id>="<plain-text>"'],
})

Again, we can generate inputs from the grammar.

In [ ]:
simple_html_fuzzer = GrammarFuzzer(SIMPLE_HTML_GRAMMAR)

In [ ]:
for i in range(20):
    fuzz_html = simple_html_fuzzer.fuzz()
    print(repr(fuzz_html))

Such inputs, of course, are great for systematic testing. Our sister book, [the fuzzing book](https://www.fuzzingbook.org/), covers these and more.

## Derivation Trees

To produce inputs from a grammar, the fuzzingbook `GrammarFuzzer` makes use of a structure called a *derivation tree*. A derivation tree encodes the individual expansion steps undertaken while producing the output.

Let us illustrate derivation trees by example, using the last HTML output we produced.

In [ ]:
fuzz_html

The `GrammarFuzzer` attribute `derivation_tree` holds the last tree used to produced this input. We can visualize the tree as follows:

In [ ]:
from fuzzingbook.GrammarFuzzer import display_tree

In [ ]:
# ignore
def display_tree(tree):
    def graph_attr(dot):
        dot.attr('node', shape='plain')
        dot.attr('node', fontname="'Fira Mono', 'Source Code Pro', 'Courier', monospace")
        
    def node_attr(dot, nid, symbol, ann):
        fuzzingbook.GrammarFuzzer.default_node_attr(dot, nid, symbol, ann)
        if symbol.startswith('<'):
            dot.node(repr(nid), fontcolor='#0060a0')
        else:
            dot.node(repr(nid), fontcolor='#00a060')
        dot.node(repr(nid), scale='2')
    
    return fuzzingbook.GrammarFuzzer.display_tree(tree,
        node_attr=node_attr,
        graph_attr=graph_attr)

In [ ]:
display_tree(simple_html_fuzzer.derivation_tree)

From top to bottom, we see that the input was constructed from a `<start>` symbol, which then expanded into `html`, which then expanded into HTML text, and so on. Multiple children in a tree stand for a concatenation of individual symbols.

Internally, these trees come as pairs `(symbol, children)`, where `symbol` is the name of a node (say, `<html>`), and `children` is a (possibly empty) list of subtrees. Here are the topmost nodes of the above tree:

In [ ]:
import pprint

In [ ]:
pp = pprint.PrettyPrinter(depth=7)
pp.pprint(simple_html_fuzzer.derivation_tree)

To produce abstract failure-inducing patterns, we will work on this very structure. The idea is to

1. systematically replace subtrees by other, generated, compatible subtrees (e.g. replace one `<html>` subtree in the concrete input by some other generated `<html>` subtree);
2. see whether these subtrees also result in failures; and
3. if they do, use the nonterminal (`<html>`) as a placeholder in the pattern.

This will involve some subtree manipulation, construction, and finally testing. First of all, though, we need to be able to turn an _existing input_ into a derivation tree.

## Parsing

The activity of creating a structure out of an unstructured input is called _parsing_. Generally speaking, a _parser_ uses a _grammar_ to create a _derivation tree_ (also called *parse tree* in parsing contexts) from a string input.

Again, there's a whole body of theory (and practice!) around constructing parsers. We make our life simple by using an existing parser (again, from [the fuzzing book](https://www.fuzzingbook.org/Parser.html)), which does just what we want. The `EarleyParser` is instantiated with a grammar such as `SIMPLE_HTML_GRAMMAR`:

In [ ]:
from fuzzingbook.Parser import EarleyParser

In [ ]:
simple_html_parser = EarleyParser(SIMPLE_HTML_GRAMMAR)

Its method `parse()` returns an iterator over multiple possible ~parse trees~ derivation trees.  (There can be multiple trees because the grammar could be ambiguous). We are only interested in the first such tree. Let us parse `BAD_INPUT` and inspect the resulting ~parse tree~ derivation tree:

In [ ]:
bad_input_tree = list(simple_html_parser.parse(BAD_INPUT))[0]

In [ ]:
display_tree(bad_input_tree)

This derivation tree has the same structure as the one created from our `GrammarFuzzer` above. We see how the `<tagged-text>` is composed of three elements:

1. an`<opening-tag>` (`<foo>`);
2. a `<html>` element which becomes `<plain-text>` (`"bar`); and
3. a `<closing-tag>` (`</foo>`).

We can easily turn the tree back into a string. The method `tree_to_string()` traverses the tree left-to-right and joins all nonterminal symbols.

In [ ]:
from fuzzingbook.GrammarFuzzer import tree_to_string, all_terminals

In [ ]:
tree_to_string(bad_input_tree)

In [ ]:
assert tree_to_string(bad_input_tree) == BAD_INPUT

## Mutating the Tree

In [ ]:
from fuzzingbook.Grammars import is_valid_grammar

In [ ]:
class TreeMutator:
    def __init__(self, grammar, tree, log=False):
        assert is_valid_grammar(grammar)
        self.grammar = grammar
        self.tree = tree
        self.log = log

In [ ]:
class TreeMutator(TreeMutator):
    def get_subtree(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return tree

        return self.get_subtree(path[1:], children[path[0]])

In [ ]:
def bad_input_tree_mutator():
    return TreeMutator(SIMPLE_HTML_GRAMMAR, bad_input_tree, log=2)    

In [ ]:
bad_input_tree_mutator().get_subtree([0, 0, 1, 0])

In [ ]:
class TreeMutator(TreeMutator):
    def new_tree(self, start_symbol):
        if self.log >= 2:
            print(f"Creating new tree for {start_symbol}")

        fuzzer = GrammarFuzzer(self.grammar, start_symbol=start_symbol)
        fuzzer.fuzz()
        return fuzzer.derivation_tree

In [ ]:
bad_input_tree_mutator().new_tree('<plain-text>')

In [ ]:
class TreeMutator(TreeMutator):
    def mutate(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return self.new_tree(node)

        head = path[0]
        new_children = (children[:head] +
                        [self.mutate(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
bad_input_tree_mutator().mutate([0])

In [ ]:
tree_to_string(bad_input_tree_mutator().mutate([0, 0, 1, 0]))

## Generalizing Trees

In [ ]:
class TreeGeneralizer(TreeMutator):
    def __init__(self, grammar, tree, test,
                 max_tries_for_abstraction=10,
                 **kwargs):
        super().__init__(grammar, tree, **kwargs)
        self.test = test
        self.max_tries_for_abstraction = max_tries_for_abstraction

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def test_tree(self, tree):
        s = tree_to_string(tree)
        if self.log:
            print(f"Testing {repr(s)}...", end="")
        try:
            self.test(s)
        except Exception as exc:
            if self.log:
                print(f"FAIL ({type(exc).__name__})")
            ret = False
        else:
            if self.log:
                print(f"PASS")
            ret = True

        return ret

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def can_generalize(self, path, tree=None):
        for i in range(self.max_tries_for_abstraction):
            mutated_tree = self.mutate(path, tree)
            if self.test_tree(mutated_tree):
                # Failure no longer occurs; cannot abstract
                return False
            
        return True

In [ ]:
def bad_input_tree_generalizer():
    return TreeGeneralizer(SIMPLE_HTML_GRAMMAR, bad_input_tree,
                           remove_html_markup, log=True)    

In [ ]:
bad_input_tree_generalizer().can_generalize([0])

In [ ]:
bad_input_tree_generalizer().can_generalize([0, 0, 1, 0])

In [ ]:
bad_input_tree_generalizer().can_generalize([0, 0, 0])

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def find_paths(self, predicate, path=None, tree=None):
        if path is None:
            path = []
        if tree is None:
            tree = self.tree
            
        node, children = self.get_subtree(path)

        if predicate(path, tree):
            if self.log:
                node, children = self.get_subtree(path)
            return [path]

        paths = []
        for i, child in enumerate(children):
            child_node, _ = child
            if child_node in self.grammar:
                paths += self.find_paths(predicate, path + [i])

        return paths        
    
    def generalizable_paths(self):
        return self.find_paths(self.can_generalize)

In [ ]:
bad_input_tree_generalizer().generalizable_paths()

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def generalize_path(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return node, []

        head = path[0]
        new_children = (children[:head] +
                        [self.generalize_path(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
all_terminals(bad_input_tree_generalizer().generalize_path([0, 0, 0]))

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def generalize(self):
        tree = self.tree
        for path in self.generalizable_paths():
            tree = self.generalize_path(path, tree)
            
        return tree

In [ ]:
all_terminals(bad_input_tree_generalizer().generalize())

## Putting it all Together

In [ ]:
from DeltaDebugger import CallCollector, is_reducible

In [ ]:
import copy

In [ ]:
class DDSetDebugger(CallCollector):
    def __init__(self, grammar, 
                 generalizer=TreeGeneralizer,
                 parser=EarleyParser,
                 **kwargs):
        super().__init__(**kwargs)
        self.grammar = grammar
        self.parser = parser(grammar)
        self.generalizer = generalizer

    def generalized_args(self, **kwargs):
        generalized_args = copy.deepcopy(self.args())

        for arg in self.args():
            def test(value):
                return self.call({arg: value})

            value = self.args()[arg]
            if is_reducible(value):
                tree = list(self.parser.parse(value))[0]
                gen = self.generalizer(self.grammar, tree, test, **kwargs)
                generalized_args[arg] = all_terminals(gen.generalize())

        return generalized_args

In [ ]:
with DDSetDebugger(SIMPLE_HTML_GRAMMAR) as dd:
    remove_html_markup(BAD_INPUT)

In [ ]:
dd.generalized_args()['s']

## More Examples

### Square Root

In [ ]:
from Assertions import square_root

In [ ]:
INT_GRAMMAR = {
    "<start>":
        ["<int>"],

    "<int>":
        ["<positive-int>", "-<positive-int>"],

    "<positive-int>":
        ["<digit>", "<nonzero-digit><positive-int>"],

    "<nonzero-digit>": list("123456789"),
    
    "<digit>": list(string.digits),
}

In [ ]:
def square_root_test(s):
    return square_root(int(s))

In [ ]:
with DDSetDebugger(INT_GRAMMAR) as dd:
    square_root_test("-1")

In [ ]:
dd.generalized_args(log=True)['s']

### Middle

In [ ]:
from StatisticalDebugger import middle

In [ ]:
XYZ_GRAMMAR = {
    "<start>":
        ["<int>, <int>, <int>"],

    "<int>":
        ["<positive-int>", "-<positive-int>"],

    "<positive-int>":
        ["<digit>", "<nonzero-digit><positive-int>"],

    "<nonzero-digit>": list("123456789"),
    
    "<digit>": list(string.digits),
}

In [ ]:
def middle_test(s):
    x, y, z = eval(s)
    assert middle(x, y, z) == sorted([x, y, z])[1]

In [ ]:
with ExpectError(AssertionError):
    middle_test("2, 1, 3")

In [ ]:
with DDSetDebugger(XYZ_GRAMMAR) as dd:
    middle_test("2, 1, 3")

In [ ]:
dd.generalized_args(log=True)['s']

# Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## _Section 1_

\todo{Add}

## _Section 2_

\todo{Add}

### Excursion: All the Details

This text will only show up on demand (HTML) or not at all (PDF). This is useful for longer implementations, or repetitive, or specialized parts.

### End of Excursion

## _Section 3_

\todo{Add}

_If you want to introduce code, it is helpful to state the most important functions, as in:_

* `random.randrange(start, end)` - return a random number [`start`, `end`]
* `range(start, end)` - create a list with integers from `start` to `end`.  Typically used in iterations.
* `for elem in list: body` executes `body` in a loop with `elem` taking each value from `list`.
* `for i in range(start, end): body` executes `body` in a loop with `i` from `start` to `end` - 1.
* `chr(n)` - return a character with ASCII code `n`

In [ ]:
import random

In [ ]:
def int_fuzzer():
    """A simple function that returns a random integer"""
    return random.randrange(1, 100) + 0.5

In [ ]:
# More code
pass

## _Section 4_

\todo{Add}

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

In [ ]:
print(int_fuzzer())

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

https://rahul.gopinath.org/post/2020/07/15/ddset/

https://rahul.gopinath.org/post/2020/08/03/simple-ddset/



_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_

## Reducing Trees

In [ ]:
from DeltaDebugger import DeltaDebugger

In [ ]:
import copy

In [ ]:
from IPython.display import display

In [ ]:
class TreeHDDReducer(TreeGeneralizer):
    def _reduce(self, path, tree):
        """This is HDD"""

        node, children = self.get_subtree(path, tree)
            
        if len(path) >= 1:
            parent, parent_children = self.get_subtree(path[:-1], tree)
 
            assert parent_children[path[-1]] == (node, children)

            def test_children(children):
                parent_children[path[-1]] = (node, children)
                s = tree_to_string(tree)
                self.test(s)

            with DeltaDebugger() as dd:
                test_children(children)
            
            # display(display_tree(tree))

            children = dd.min_args()['children']
            parent_children[path[-1]] = (node, children)
        
        for i, child in enumerate(children):
            self._reduce(path + [i], tree)
            
        return tree

    def reduce(self):
        return self._reduce([], self.tree)

In [ ]:
def bad_input_tree_hdd_reducer():
    return TreeHDDReducer(SIMPLE_HTML_GRAMMAR, copy.deepcopy(bad_input_tree),
                       remove_html_markup, log=True)    

In [ ]:
all_terminals(bad_input_tree_hdd_reducer().reduce())

## More Reducing Trees

In [ ]:
class TreeReducer(TreeGeneralizer):
    def new_min_tree(self, start_symbol):
        if self.log >= 2:
            print(f"Creating new minimal tree for {start_symbol}")

        fuzzer = GrammarFuzzer(self.grammar, start_symbol=start_symbol,
                               min_nonterminals=0,
                               max_nonterminals=0)
        fuzzer.fuzz()
        return fuzzer.derivation_tree

In [ ]:
def bad_input_tree_reducer():
    return TreeReducer(SIMPLE_HTML_GRAMMAR, bad_input_tree,
                       remove_html_markup, log=2)    

In [ ]:
tree_to_string(bad_input_tree_reducer().new_min_tree('<start>'))

In [ ]:
class TreeReducer(TreeReducer):
    def reduce_path(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return self.new_min_tree(node)

        head = path[0]
        new_children = (children[:head] +
                        [self.reduce_path(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
tree_to_string(bad_input_tree_reducer().reduce_path([0, 0, 1, 0]))

In [ ]:
class TreeReducer(TreeReducer):
    def can_reduce(self, path, tree=None):
        reduced_tree = self.reduce_path(path, tree)
        if self.test_tree(reduced_tree):
            # Failure no longer occurs; cannot reduce
            return False

        return True

In [ ]:
class TreeReducer(TreeReducer):
    def reducible_paths(self):
        return self.find_paths(self.can_reduce)

In [ ]:
bad_input_tree_reducer().reducible_paths()

In [ ]:
class TreeReducer(TreeReducer):
    def reduce(self):
        tree = self.tree
        for path in self.reducible_paths():
            tree = self.reduce_path(path, tree)
            
        return tree

In [ ]:
all_terminals(bad_input_tree_reducer().reduce())